<h2>Construct keyfiles from project directory containing a Base FVS Rx template.</h2>

In [1]:
import os
import glob
from jinja2 import Template
import pandas as pd
import random

Create a jinja2 template from a Base_Rx.key file.

In [2]:
# FOR LONG ROTATIONS
rotation = "LONG"
# read in the base_rx keyfile template using jinja2 templating
with open(os.path.join('Rx_Template','LongRotation.key'), 'r') as base_keyfile:
    template = Template(base_keyfile.read())
    print('Found LongRotation.key and created jinja2 template.')

Found LongRotation.key and created jinja2 template.


In [3]:
# # FOR SHORT ROTATIONS
# rotation = "SHORT"
# # read in the base_rx keyfile template using jinja2 templating
# with open(os.path.join('Rx_Template','ShortRotation.key'), 'r') as base_keyfile:
#     template = Template(base_keyfile.read())
#     print('Found ShortRotation.key and created jinja2 template.')

A dictionary for holding the items to insert into an FVS keyfile template using jinja2 templating.

In [4]:
inserts = {}

Specify the FVS output database for insertion in the jinja2 template

In [5]:
inserts['FVSOut'] = 'Rotations'

Look in the Rx_Template directory, add all the kcp files to the `inserts` dictionary.

In [6]:
template_kcps = glob.glob(os.path.join('Rx_Template','*.kcp'))
if len(template_kcps) > 0:
    print('Found the following kcp files in the Rx_Template directory:')
    for kcp in template_kcps:
        fname = os.path.split(kcp)[-1]
        print(fname, end='...')
        # read the kcp file
        key = fname.split('.')[0] # key for item in inserts dictionary
        with open(kcp, 'r') as item:
            value = item.read()
        # add the contents of the kcp file to the inserts dictionary
        inserts[key] = value
        print(' added to template.')
else: 
    raise FileNotFoundError('No kcp files found in the Rx_Template directory.')

Found the following kcp files in the Rx_Template directory:
Mgmt_Costs.kcp... added to template.


Gather variant-specific KCP files for SDImax and Log_Values

In [7]:
SDImax_dict = {} # variant: SDImax keywords
logvalue_dict = {} # variant: log value specs

variant_kcps = glob.glob(os.path.join('Rx_Template','variant_specific_kcps','*.kcp'))
if len(variant_kcps) > 0:
    print('Found the following kcp files in the variant_specific_kcps directory:')
    for kcp in variant_kcps:
        fname = os.path.split(kcp)[-1]
        variant = fname.split('.')[0][-2:]
        kcp_type = fname.split('.')[0][:-3]
        print(variant, kcp_type, end='...')
        # read the kcp file
        with open(kcp, 'r') as item:
            key = variant
            value = item.read()
        if kcp_type == 'SDImax':
            SDImax_dict[key] = value
        elif kcp_type == 'logvalue':
            logvalue_dict[key] = value
        print(' added to template.')
else: 
    raise FileNotFoundError('No kcp files found in the variant_specific_kcps directory.')

Found the following kcp files in the variant_specific_kcps directory:
OP logvalue... added to template.
PN logvalue... added to template.
WC logvalue... added to template.
OP SDImax... added to template.
PN SDImax... added to template.
WC SDImax... added to template.


A function to use for creating keyfiles.

In [8]:
def create_keyfile(variant, site_index):
    '''
    Creates a single FVS keyfile based on the jinja2 template.
    '''
    inserts['SDImax'] = SDImax_dict[variant]
    inserts['Log_Value'] = logvalue_dict[variant]
    inserts['site_index'] = site_index
    inserts['variant'] = variant
    # FVS slows down outputting to large databases, so we'll divide output among 10 databases
    #inserts['db_num'] = random.randint(1,10) # add a random number, 1-10 for a output database suffix
    
    fname = 'fvs'+variant+'_DFsite'+str(site_index)+'_'+rotation+'.key'
    path = os.path.abspath('keyfiles_to_run')
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)
    with open(os.path.join('keyfiles_to_run',fname),'w') as keyfile:
        keyfile.write(template.render(**inserts))

In [9]:
# range of site index values to use for OP variant
DF50 = [x for x in range(50, 165, 5)] # Douglas-fir 50-year Site Index

Create the keyfiles!

In [10]:
for site in DF50:
    create_keyfile('OP', site)